In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import nibabel
import random
import sys

import tensorflow.keras
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Activation, concatenate, Conv2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2DTranspose
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda
from tensorflow.keras.layers import add, concatenate, UpSampling2D, Concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import History

import tensorflow as tf

import sklearn
from sklearn.model_selection import train_test_split

import skimage
from skimage import img_as_ubyte, img_as_float32, img_as_uint
import skimage.transform
from skimage.transform import resize
import skimage.io
from skimage.io import imread, imshow, imread_collection, concatenate_images, imsave
from skimage.segmentation import mark_boundaries
from skimage.exposure import rescale_intensity
from skimage import io
from scipy import ndimage as ndi
from skimage.morphology import label, ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from skimage.io import imread
import scipy.misc

import cv2

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from glob import glob

Load Data Functions

In [ ]:
def load_train_data():
    imgs_train = np.load('imgs_train.npy')
    masks_train = np.load('masks_train.npy')
    return imgs_train, masks_train

def load_test_data():
    imgs_test = np.load('imgs_test.npy')
    return imgs_test

Coefficients

In [ ]:
img_rows = int(512/2)
img_cols = int(512/2)
smooth = 1e-5

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def jaccard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(K.abs(y_true_f * y_pred_f))
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def jaccard_coef_loss(y_true, y_pred):
  return 1-jaccard_coef(y_true, y_pred)

2 UNet Model

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Nadam

def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.15)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.15)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.15)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.15)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid', kernel_regularizer=regularizers.l2(1e-4))(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Nadam(lr = 2e-4), loss=dice_coef_loss, metrics=[dice_coef]) # Adam(learning_rate=1e-3)

    return model

def get_unet2():
    inputs = Input((img_rows, img_cols, 1))

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.15)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.15)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(conv3)
    conv3 = BatchNormalization()(conv3)

    # Decoder
    up4 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)

    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv5)

    model = Model(inputs=[inputs], outputs=[outputs])

    model.compile(optimizer=Adam(learning_rate=1e-3), loss=dice_coef_loss, metrics=[dice_coef]) # Adam(learning_rate=1e-3)

    return model

FCN Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.applications.vgg16 import *

def fcn():
    ch_out = 1
    inputs = Input((img_rows, img_cols, 1))
    expanded_inputs = Concatenate()([inputs, inputs, inputs])

    # Building a pre-trained VGG-16 feature extractor (i.e., without the final FC layers)
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=expanded_inputs)
    # Recovering the feature maps generated by each of the 3 final blocks:
    f3 = vgg16.get_layer('block3_pool').output
    f4 = vgg16.get_layer('block4_pool').output
    f5 = vgg16.get_layer('block5_pool').output

    # Replacing VGG dense layers by convolutions:
    f5_conv1 = Conv2D(filters=4086, kernel_size=7, padding='same',
                      activation='relu', kernel_regularizer=regularizers.l2(1e-4))(f5)
    f5_drop1 = Dropout(0.5)(f5_conv1)
    f5_conv2 = Conv2D(filters=4086, kernel_size=1, padding='same',
                      activation='relu', kernel_regularizer=regularizers.l2(1e-4))(f5_drop1)
    f5_drop2 = Dropout(0.5)(f5_conv2)
    f5_conv3 = Conv2D(filters=ch_out, kernel_size=1, padding='same',
                      activation=None, kernel_regularizer=regularizers.l2(1e-4))(f5_drop2)


    # Using a transposed conv (w/ s=2) to upscale `f5` into a 14 x 14 map
    # so it can be merged with features from `f4_conv1` obtained from `f4`:
    f5_conv3_x2 = Conv2DTranspose(filters=ch_out, kernel_size=4, strides=2,
                                use_bias=False, padding='same', activation='relu')(f5)
    f4_conv1 = Conv2D(filters=ch_out, kernel_size=1, padding='same',
                      activation=None, kernel_regularizer=regularizers.l2(1e-4))(f4)

    # Merging the 2 feature maps (addition):
    merge1 = add([f4_conv1, f5_conv3_x2])

    # We repeat the operation to merge `merge1` and `f3` into a 28 x 28 map:
    merge1_x2 = Conv2DTranspose(filters=ch_out, kernel_size=4, strides=2,
                                use_bias=False, padding='same', activation='relu')(merge1)
    f3_conv1 = Conv2D(filters=ch_out, kernel_size=1, padding='same',
                      activation=None, kernel_regularizer=regularizers.l2(1e-4))(f3)
    merge2 = add([f3_conv1, merge1_x2])

    # Finally, we use another transposed conv to decode and up-scale the feature map
    # to the original shape, i.e., using a stride 8 to go from 28 x 28 to 224 x 224 here:
    outputs = Conv2DTranspose(filters=ch_out, kernel_size=16, strides=8,
                              padding='same', activation=None)(merge2)

    fcn_model = Model(inputs, outputs)

    fcn_model.compile(optimizer=Adam(learning_rate=1e-3), loss=dice_coef_loss, metrics=[dice_coef])

    return fcn_model

2-D VNet Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,PReLU,Conv2DTranspose,add,concatenate,Input,Dropout,BatchNormalization,Activation
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

def resBlock(conv,stage,keep_prob,stage_num=5):#收缩路径
    inputs=conv

    for _ in range(3 if stage>3 else stage):
        conv=PReLU()(BatchNormalization()(Conv2D(16*(2**(stage-1)), 5, activation = None, padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(conv)))
        
    conv_add=PReLU()(add([inputs,conv]))
    conv_drop=Dropout(keep_prob)(conv_add)

    if stage<stage_num:
        conv_downsample=PReLU()(BatchNormalization()(Conv2D(16*(2**stage), 2, strides=(2, 2),activation = None, padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(conv_drop)))
        return conv_downsample,conv_add
    else:
        return conv_add,conv_add

def up_resBlock(forward_conv,input_conv,stage):

    conv=concatenate([forward_conv,input_conv],axis = -1)
    print('conv_concatenate:',conv.get_shape().as_list())
    for _ in range(3 if stage>3 else stage):
        conv=PReLU()(BatchNormalization()(Conv2D(16*(2**(stage-1)), 5, activation = None, padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(conv)))
        print('conv_up_stage_%d:' %stage,conv.get_shape().as_list())
    conv_add=PReLU()(add([input_conv,conv]))
    if stage>1:
        conv_upsample=PReLU()(BatchNormalization()(Conv2DTranspose(16*(2**(stage-2)),2,strides=(2, 2),padding='valid',activation = None,kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(conv_add)))
        return conv_upsample
    else:
        return conv_add

def vnet(pretrained_weights = None,input_size = (256,256,1),num_class=1,is_training=True,stage_num=5,thresh=0.5):
    keep_prob = 0.5 if is_training else 1.0 # keep_prob --> dropout
    features=[]
    input_model = Input(input_size)
    x=PReLU()(BatchNormalization()(Conv2D(16, 5, activation = None, padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(input_model)))

    for s in range(1,stage_num+1):
        x,feature=resBlock(x,s,keep_prob,stage_num)
        features.append(feature)

    conv_up=PReLU()(BatchNormalization()(Conv2DTranspose(16*(2**(s-2)),2,strides=(2, 2),padding='valid',activation = None,kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(x)))

    for d in range(stage_num-1,0,-1):
        conv_up=up_resBlock(features[d-1],conv_up,d)
    if num_class>1:
        conv_out=Conv2D(num_class, 1, activation = 'softmax', padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(conv_up)
    else:
        conv_out=Conv2D(num_class, 1, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal', kernel_regularizer=regularizers.l2(1e-4))(conv_up)

    model=Model(inputs=input_model,outputs=conv_out)
    print(model.output_shape)

    model.compile(optimizer = Nadam(lr = 2e-4), loss=dice_coef_loss, metrics=[dice_coef])

    return model

Preprocess

In [ ]:
img_rows = int(512/2)
img_cols = int(512/2)

def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

Training

In [ ]:
import imageio.core.util

def ignore_warnings(*args, **kwargs):
    pass

imageio.core.util._precision_warn = ignore_warnings

print('-'*30)
print('Loading and preprocessing train data...')
print('-'*30)
imgs_train, imgs_mask_train = load_train_data()

imgs_train = preprocess(imgs_train)
imgs_mask_train = preprocess(imgs_mask_train)

imgs_train = imgs_train.astype('float32')
imgs_mask_train = imgs_mask_train.astype('float32')

print('-'*30)
print('Creating and compiling model...')
print('-'*30)
model = vnet() # CALL MODEL
model_checkpoint = ModelCheckpoint('weights_vnet.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

print('-'*30)
print('Fitting model...')
print('-'*30)
history=model.fit(imgs_train, imgs_mask_train, batch_size=1, epochs=30, verbose=1, shuffle=True,
            validation_split=0.3,
            callbacks=[model_checkpoint])

print('-'*30)
print('Loading and preprocessing test data...')
print('-'*30)

imgs_test = load_test_data()
imgs_test = preprocess(imgs_test)
imgs_test = imgs_test.astype('float32')

print('-'*30)

Predict

In [ ]:
print('Loading saved weights...')
print('-'*30)
model = get_unet() # get_unet()
model.load_weights('YOUR_MODEL_WEIGHT_PATH')

model_vnet = vnet()
model_vnet.load_weights('YOUR_MODEL_WEIGHT_PATH')

print('-'*30)
print('Predicting masks on test data...')
print('-'*30)
imgs_mask_test = model.predict(imgs_train, verbose=1) # imgs_test
imgs_mask_test_vnet = model_vnet.predict(imgs_train, verbose=1)

np.save('/media/kursat/TOSHIBA EXT5/projects/ANN_PROJE/imgs_mask_test_trainData.npy', imgs_mask_test)
np.save('/media/kursat/TOSHIBA EXT5/projects/ANN_PROJE/imgs_mask_test_trainData_vnet.npy', imgs_mask_test_vnet)
print('-' * 30)
print('Saving predicted masks to files...')
print('-' * 30)

imgs_test_vnet = model_vnet.predict(imgs_test, verbose=1)
np.save('/media/kursat/TOSHIBA EXT5/projects/ANN_PROJE/imgs_test_vnet.npy', imgs_test_vnet)
print('-' * 30)
print('Saving predicted masks to files (test data)...')
print('-' * 30)

Plot

In [ ]:
"""
Train data
"""

train_data = np.load('YOUR_TRAIN_DATA_PATH')
ground_truth = np.load('YOUR_MASK_DATA_PATH')
prediction = np.load('YOUR_PREVIOUS_PREDICTION_DATA_PATH')
prediction_vnet = np.load('YOUR_PREDICTION_DATA_PATH')

channel = 452 # write any number

plt.subplot(141)
plt.imshow(train_data[channel, :, :], cmap='gray')
plt.title("Train Data")

plt.subplot(142)
plt.imshow(ground_truth[channel, :, :], cmap='gray')
plt.title("Ground Truth")

plt.subplot(143)
plt.imshow(prediction[channel, :, :], cmap='gray')
plt.title("Initial Result: UNet", fontsize=10)

plt.subplot(144)
plt.imshow(prediction_vnet[channel, :, :], cmap='gray')
plt.title("Third Report: VNet", fontsize=10)
plt.show()

In [ ]:
"""
Test Data
"""

test_data = np.load('YOUR_TEST_DATA_PATH')
prediction = np.load('OUR_PREVIOUS_PREDICTION_TEST_DATA_PATH')
prediction_vnet = np.load('YOUR_PREDICTION_TEST_DATA_PATH')

channel = 115 # write any number

plt.subplot(131)
plt.imshow(test_data[channel, :, :], cmap='gray')
plt.title("Test Data")

plt.subplot(132)
plt.imshow(prediction[channel, :, :], cmap='gray')
plt.title("Initial Result: UNet", fontsize=10)

plt.subplot(133)
plt.imshow(prediction_vnet[channel, :, :], cmap='gray')
plt.title("Third Report: VNet", fontsize=10)
plt.show()